In [307]:
# Dependencies
import pandas as pd

In [308]:
# # Modules
# import os
# import csv

In [309]:
# Load in file
student_file = "../Resources/students_complete.csv"
schools_file = "../Resources/schools_complete.csv"

In [310]:
# Open the students file in CSV
student_df = pd.read_csv(student_file)
student_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [311]:
# Open the schools file in CSV
schools_df = pd.read_csv(schools_file)
schools_df.head()

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [312]:
# Count total number of schools in district
total_schools = len(student_df["school_name"].unique())
total_schools

15

In [313]:
# Count total number of students
student_count = len(student_df)
student_count

39170

In [314]:
# Sum total budget
total_budget = schools_df["budget"].sum()
total_budget

24649428

In [315]:
# Calculate average math and reading scores
average_math = student_df["math_score"].mean()
average_reading = student_df["reading_score"].mean()

In [316]:
# Calculate number of of students passing for math and reading    
math_passing_df = student_df.loc[student_df["math_score"] >= 65]
count_math = len(math_passing_df)
reading_passing_df = student_df.loc[student_df["reading_score"] >= 65]
count_reading = len(reading_passing_df)

# Divide passing scores for math and reading by total student count
percentage_passing_math = count_math/student_count
percentage_passing_reading = count_reading/student_count
s = math_passing_df.groupby("school_name")    
s.groupby

AttributeError: 'DataFrameGroupBy' object has no attribute 'groupby'

In [317]:
overall = student_df[(student_df["math_score"] >= 65) & (student_df["reading_score"] >= 65)].count()["Student ID"]
# overall_count = len(overall)
overall_passing = overall / student_count


In [318]:
student_df[student_df["math_score"]>=65].head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
4,4,Bonnie Ray,F,9th,Huang High School,97,84
5,5,Bryan Miranda,M,9th,Huang High School,94,94
6,6,Sheena Carter,F,11th,Huang High School,82,80
7,7,Nicole Baker,F,12th,Huang High School,96,69


In [319]:
district_df = pd.DataFrame({
    "Total Schools":[total_schools],
    "Total Students":f'{student_count:,}',
    "Total Budget":f'${total_budget:,}',
    "Average Math Score":[average_math],
    "Average Reading Score":[average_reading],
    "% Passing Math":f'{percentage_passing_math:.2%}',
    "% Passing Reading":f'{percentage_passing_reading:.2%}',
    "% Overall Passing":f'{overall_passing:.2%}'
})
district_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428",78.985371,81.87784,84.73%,96.20%,81.77%


In [320]:
school_summary_df = schools_df[["school_name", "type", "size","budget"]]
school_summary_df.head()
school_summary_df["Per Student Budget"] = school_summary_df["budget"]/school_summary_df["size"]
school_summary_df["Per Student Budget"].head()

0    655.0
1    639.0
2    600.0
3    652.0
4    625.0
Name: Per Student Budget, dtype: float64

In [324]:
# student_by_school_average = student_df["reading_score"].mean()
group_schools = student_df.groupby("school_name")

# Get the average of each column within the GroupBy object
schools_average_df = group_schools[["reading_score", "math_score"]].mean()
schools_average_df 

,reading_score,math_score
school_name,,
Bailey High School,81.033963,77.048432
Cabrera High School,83.975780,83.061895
Figueroa High School,81.158020,76.711767
Ford High School,80.746258,77.102592
Griffin High School,83.816757,83.351499
Hernandez High School,80.934412,77.289752
Holden High School,83.814988,83.803279
Huang High School,81.182722,76.629414
Johnson High School,80.966394,77.072464


In [328]:
passingReadCount = student_df[student_df['reading_score'] >= 65].groupby("school_name")['student_name'].count().reset_index()
passingReadCount

,school_name,student_name
0,Bailey High School,4705
1,Cabrera High School,1858
2,Figueroa High School,2788
3,Ford High School,2571
4,Griffin High School,1468
5,Hernandez High School,4385
6,Holden High School,427
7,Huang High School,2756
8,Johnson High School,4498
9,Pena High School,962


In [329]:
passingMathCount = student_df[student_df['math_score'] >= 65].groupby("school_name")['student_name'].count().reset_index()
passingMathCount

,school_name,student_name
0,Bailey High School,3877
1,Cabrera High School,1858
2,Figueroa High School,2276
3,Ford High School,2142
4,Griffin High School,1468
5,Hernandez High School,3603
6,Holden High School,427
7,Huang High School,2267
8,Johnson High School,3712
9,Pena High School,962


In [331]:
passingOverall = student_df[(student_df['math_score'] >= 65) & (student_df['reading_score'] >=65)].groupby("school_name")['student_name'].count().reset_index()
passingOverall

,school_name,student_name
0,Bailey High School,3670
1,Cabrera High School,1858
2,Figueroa High School,2148
3,Ford High School,2015
4,Griffin High School,1468
5,Hernandez High School,3409
6,Holden High School,427
7,Huang High School,2137
8,Johnson High School,3511
9,Pena High School,962


In [323]:
# def get_passing_math(school):
#     total_students = len(student_df.loc[student_df.school_name == school])
#     total_students_passing_math = len(student_df.loc[(student_df.school_name == school) & (student_df.math_score >= 65)]) 

#     return total_students_passing_math / total_students

# schools = renamed_data({"School Name"})
# schools
# % Passing Math (The percentage of students that passed math.)
# total_bailey_students = len(student_df.loc[student_df.school_name == "Bailey High School"])
# # student_df.loc[student_df.school_name == "Bailey High School"]
# total_bailey_students_passing_math = len(student_df.loc[(student_df.school_name == "Bailey High School") & (student_df.math_score >= 65)])

# total_bailey_students_passing_math / total_bailey_students



# math_passing_df = student_df.loc[student_df["math_score"] >= 65]

# math_passing_by_school_df = student_df.loc[student_df["math_score"] >= 65,["math_score"]]
# count_math = len(math_passing_by_school_df)

# group_schools = student_df.groupby(["school_name","math_score"])
# Get the average of each column within the GroupBy object
# math_passing_by_school_df = group_schools.loc[group_schools["math_score"] >= 65,["math_score"]]
# math_passing_by_school_df.head(100)


TypeError: 'DataFrame' object is not callable

In [234]:
combined_data = pd.merge(school_summary_df,schools_average_df,how="outer",on="school_name")
renamed_data = combined_data.rename(
    columns={"school_name": "School Name",
            "type": "School Type",
            "size":"School Size",
            "budget":"School Budget",
            "reading_score":"Average Reading Score",
            "math_score":"Average Math Score"})
renamed_data.head(15)
# % Passing Math (The percentage of students that passed math.)
# % Passing Reading (The percentage of students that passed reading.)
# % Overall Passing (The percentage of students that passed math and reading.)

,School Name,School Type,School Size,School Budget,Per Student Budget,Average Reading Score,Average Math Score
0,Huang High School,District,2917,1910635,655.0,81.182722,76.629414
1,Figueroa High School,District,2949,1884411,639.0,81.158020,76.711767
2,Shelton High School,Charter,1761,1056600,600.0,83.725724,83.359455
3,Hernandez High School,District,4635,3022020,652.0,80.934412,77.289752
4,Griffin High School,Charter,1468,917500,625.0,83.816757,83.351499
5,Wilson High School,Charter,2283,1319574,578.0,83.989488,83.274201
6,Cabrera High School,Charter,1858,1081356,582.0,83.975780,83.061895
7,Bailey High School,District,4976,3124928,628.0,81.033963,77.048432
8,Holden High School,Charter,427,248087,581.0,83.814988,83.803279
9,Pena High School,Charter,962,585858,609.0,84.044699,83.839917


In [333]:
df = pd.DataFrame(columns=['SchoolName','Score'])
df

,SchoolName,Score


In [334]:
def get_passing_math(school):
    total_students = len(student_df.loc[student_df.school_name == school])
    total_students_passing_math = len(student_df.loc[(student_df.school_name == school) & (student_df.math_score >= 65)]) 

    return total_students_passing_math / total_students

for school in renamed_data["School Name"]:
    total = get_passing_math(school)
    tmp = {"SchoolName":school, "Score": total}
    df = df.append(tmp, ignore_index = True)
    print(f'{school} {total}')
    

df
# shelton = student_df.loc[student_df.school_name == 'Shelton High School']

# shelton.min()


Huang High School 0.7771683236201576
Figueroa High School 0.7717870464564259
Shelton High School 1.0
Hernandez High School 0.7773462783171521
Griffin High School 1.0
Wilson High School 1.0
Cabrera High School 1.0
Bailey High School 0.7791398713826366
Holden High School 1.0
Pena High School 1.0
Wright High School 1.0
Rodriguez High School 0.7794448612153039
Johnson High School 0.7796681369460198
Ford High School 0.7820372398685652
Thomas High School 1.0


,SchoolName,Score
0,Huang High School,0.777168
1,Figueroa High School,0.771787
2,Shelton High School,1.000000
3,Hernandez High School,0.777346
4,Griffin High School,1.000000
5,Wilson High School,1.000000
6,Cabrera High School,1.000000
7,Bailey High School,0.779140
8,Holden High School,1.000000
9,Pena High School,1.000000
